In [1]:
from controller import DMC
import control as ctrl
import numpy as np
import plotly.graph_objects as go
import sympy
sympy.init_printing(use_latex=True)


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [59]:
s = ctrl.TransferFunction.s
Gc = 4/((1.25*s+1)**2)
tc, yc = ctrl.step_response(Gc)

t_ss = ctrl.step_info(Gc, SettlingTimeThreshold=0.01)['SettlingTime']
T_s = t_ss/80
Nss = int(np.ceil(t_ss/T_s))
# Nss = 84
T_s, Nss

In [60]:
T_s = 0.2
Nss*T_s

In [61]:
Gd = ctrl.sample_system(Gc, T_s, method='zoh')
td, yd = ctrl.step_response(Gd)

fig = go.Figure(data=go.Scatter(x=tc, y=yc, mode='lines', name='G(s)'))
fig.add_trace(go.Scatter(x=td, y=yd, mode='markers', name='G(z)'))
fig.update_layout(
    title="Resposta ao Degrau",
    xaxis_title="t [s]",
    yaxis_title="y"
)
fig.update_layout(xaxis_range=[0, Nss*T_s])
fig.show()


In [62]:
ctrl.tf2ss(Gd)

StateSpace(array([[ 1.70428758, -0.72614904],
       [ 1.        ,  0.        ]]), array([[1.],
       [0.]]), array([[0.04605282, 0.04139302]]), array([[0.]]), 0.2)

In [63]:
ctrl.tf2ss(Gc)


StateSpace(array([[-1.6 , -0.64],
       [ 1.  ,  0.  ]]), array([[1.],
       [0.]]), array([[0.  , 2.56]]), array([[0.]]))

In [67]:
A, B, C, D = ctrl.ssdata(Gd)
N1 = 1
N2 = 15
N = N2 - N1 + 1
Nu = 5
lamda = 0.5
# Nss = 100
controller = DMC(N1=N1, N2=N2, Nu=Nu, Nss=Nss, lamda=lamda, T_s=T_s, model=yd[1:])

constraints = {
    "u_max": 2,
    "u_min": -2,
    # "delta_u_max": 1,
    # "delta_u_min": -1,
    # "y_max": 3,
    # "y_min": -3,
}

x0 = np.zeros((A.shape[0], 1))
u = 0 
du = 0
k = np.arange(0, Nss + 500, 1)

y_t = C@x0
u_k = np.array([u])
du_k = np.array([0])
t = np.array([0])
r = 0*k + 1
r[100+Nss:] = 3
r[200+Nss:] = -2
r[300+Nss:] = 0
r[400+Nss:] = -3
# 
x=x0
y=(C@x0)[0]

for i in (k):
    
    if i > Nss:
        if i > max(k) - N:
            du = controller.get_u(y, r[i], constraints)[0]
        else:
            du = controller.get_u(y, np.array([r[i:i+N]]).T, constraints)[0]  
        u = u + du

    x_k = A @ x + B*u
    y = C @ x

    x = x_k
    
    y_t = np.hstack((y_t, y))
    u_k = np.hstack((u_k, u))
    du_k = np.hstack((du_k, du))

fig = go.Figure()
fig.add_trace(go.Scatter(x=k*T_s, y=y_t[0][:-1], mode='lines', name='y'))
fig.add_trace(go.Scatter(x=k*T_s, y=r, mode='lines', name='r', line=dict(dash='dot', color='gray')))
fig.add_trace(go.Scatter(x=k*T_s, y=u_k, mode='lines', name='u'))
fig.add_trace(go.Scatter(x=k*T_s, y=du_k, mode='lines', name='du'))
fig.update_layout(
    # xaxis_title="t [s]",
    yaxis_title="y"
)
fig.update_layout(xaxis_range=[Nss*T_s, (Nss + 500)*T_s])
fig.show()